# Nature Conservancy Fish Classification - BB Crops

### Imports & Environment

In [1]:
import os
import ujson as json
import PIL
import random
import matplotlib.pyplot as plt

from glob import glob
from collections import defaultdict

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

In [2]:
# paths
data_path = DATA_HOME_DIR + '/' 
full_train_path = data_path + 'train_full/'
crop_path = data_path + 'cropped/'

# data
classes = ["ALB", "BET", "DOL", "LAG", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

### Cropping Images to Bounding Box Coordinates

So, because a few fish have been relabeled in the training set, the classes won't line up exactly with the classes in the annotation files. 

As a roundabout way of getting around this, I create a dictionary mapping image files with their respective classes so that when I iterate through the annotation files I can pair them up on the fly. 

In [3]:
class_dict = defaultdict(str)

for fp in glob(full_train_path + '*/*g'):
    cls = fp.split('/')[-2]
    im = fp.split('/')[-1]
    class_dict[im] = cls
    
print("Image Records:", len(class_dict.keys()))

Image Records: 3777


Then I iterate through each class, grab the annotations for that class, crop the image down to a square around the fish, and save it to my cropped data directory. 

In [4]:
for c in classes:
    
#     if c == "NoF":  # no annotations for fish-less images, so we do a random crop
#         fns = glob(full_train_path + c + '/*g')
#         for fn in fns:
#             im = PIL.Image.open(fn)
            
#             ht = random.uniform(96, 336)
#             wt = random.uniform(96, 336)
#             mx_dim = max([ht, wt])
#             x = random.uniform(8, im.size[0]-336)
#             y = random.uniform(8, im.size[1]-336)
            
#             cropped = im.crop((x, y, x + mx_dim, y + mx_dim))
#             cropped.save(fn.replace(full_train_path, crop_path)) 
#         continue

    
    j = json.load(open('bb_annotations/{}.json'.format(c.lower()), 'r'))
    
    for l in j: 
        if 'annotations' in l.keys() and len(l['annotations']) > 0:
            fn = l['filename'].split('/')[-1]
            cls = class_dict[fn]
            im = PIL.Image.open('{0}{1}/{2}'.format(full_train_path, cls, fn))
            anno = sorted(l['annotations'], key=lambda x: x['height']*x['width'])[-1]
            
            mx_dim = max([anno['width'], anno['height']])           
            x = anno['x']
            y = anno['y']

            cropped = im.crop((x, y, x + mx_dim, y + mx_dim))
            cropped.save('{0}{1}/{2}'.format(crop_path, cls, fn))
        else:
            print(c, l['filename'])

ALB img_07008.jpg
ALB img_06460.jpg
ALB img_04798.jpg
ALB img_02292.jpg
ALB img_01958.jpg
ALB img_00576.jpg
ALB img_00568.jpg
ALB img_00425.jpg
BET img_00379.jpg
DOL img_06773.jpg
DOL img_05444.jpg
SHARK ../data/train/SHARK/img_06082.jpg
YFT ../data/train/YFT/img_04558.jpg
YFT ../data/train/YFT/img_03183.jpg
YFT ../data/train/YFT/img_02785.jpg


In [5]:
crops = glob(crop_path + '*/*g')

print("Cropped Image Records:", len(crops))

Cropped Image Records: 3297


Looks like a few records were lost in the process (in addition to the ones printed out above), but after looking through some of the missing annotations, there's generally a good reason, such as multiple fish or obscured fish. 